# Variational Autoencoders

Variational autoencoders view autoencoding from a statistical perspective. Like classical autoencoders, they encode a dataset into a lower dimensional latent space. Additionally, though, variational autoencoders constrain the encoded vectors to roughly follow a probability distribution, e.g. a normal distribution. Here’s an example of a variational autoencoder for the same 1D sequence to sequence monochromatic signal encoding problem.

https://mlgeophysics.github.io/community/projects/auto-encoder/vae/

## Variational Autoencoder¶

Much of this code is from https://blog.keras.io/building-autoencoders-in-keras.html and https://github.com/keras-team/keras/blob/master/examples/variational_autoencoder.py

Tutorials can be found at https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf, https://jaan.io/what-is-variational-autoencoder-vae-tutorial/, and https://arxiv.org/abs/1606.05908

This notebook uses the same toy problem as the autoencoding notebook. Here we demonstrate the use of a variational autoencoder.


In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import scipy.stats

import tensorflow as tf

#from keras.models import Input 
from keras.models import Model
from keras.models import load_model

from keras.layers import Dense, LeakyReLU, Lambda, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.losses import mse
from keras import backend as K

from sklearn.model_selection import train_test_split

# Init Fidle environment
import fidle


2023-03-23 18:37:33.980208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [3]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [4]:
import tensorflow._api.v2.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
tf.config.run_functions_eagerly(False)

In [6]:
run_id, run_dir, datasets_dir = fidle.init('MNIST1')

<br>**FIDLE - Environment initialization**

Version              : 2.0b56
Run id               : MNIST1
Run dir              : ./run/MNIST1
Datasets dir         : /Users/sylvie/Documents/fidle-tp/datasets-fidle
Start time           : 23/03/23 18:37:35
Hostname             : imacdagoret.lal.in2p3.fr (Darwin)
Tensorflow log level : Warning + Error  (=1)
Update keras cache   : False
Save figs            : ./run/MNIST1/figs (False)
tensorflow           : 2.12.0
numpy                : 1.23.5
sklearn              : 1.2.2
matplotlib           : 3.7.1
pandas               : 1.5.3


In [7]:
from sklearn.model_selection import train_test_split
import h5py

## generate training, test, and validation data

In [8]:
hf = h5py.File('SLspectra.hdf5', 'r') 
data_in = hf['flambda'][:]
wl_in = hf['wl'][:]
target = hf['quantile'][:]
print(hf.keys())

<KeysViewHDF5 ['flambda', 'num', 'quantile', 'redshift', 'wl']>


In [9]:
data_in.shape

(550, 10000)

In [10]:
target.shape

(550,)

In [11]:
NWL = data_in.shape[1]
NSAMPL = data_in.shape[0]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(data_in,
                                          target,
                                          test_size=0.2)

In [13]:
# specify training parameters and callback functions

# batch size for stochastic solver  
batch_size = 16

# number of times entire dataset is considered in stochastic solver
epochs = 50

# unique name for the network for saving
unique_name = "NN_VAE_V0"
model_filename = 'model' + unique_name+'.h5'



# training history file name
history_filename = 'results_'+unique_name+'.npz'

# stop early after no improvement past epochs=patience and be verbose
earlystopper = EarlyStopping(patience=100, verbose=1)

# checkpoint and save model when improvement occurs 
checkpointer = ModelCheckpoint(model_filename, verbose=1, save_best_only=True)

# consolidate callback functions for convenience 
callbacks = [earlystopper, checkpointer]

Now things get a bit different from a vanilla autoencoder. First, we set the dimensions of the latent space. For this example we can get away with only one dimension. Intuitively, since the only difference between training examples is the phase, we only need one to encode one dimension.

In [14]:
# encoding dimension; i.e. dimensionality of the latent space
encoding_dim=5

Next we define a function to draw samples from a Gaussian, given the mean and standard deviation. We sample to encode in the latent space. Further, the way this function is defined, it lets us use backpropagation on the mean and standard deviation, even though there's a probabilistic element to this operation (this is the "reparameterization trick").

In [15]:
# define a function to sample from gaussian, given mean and log variance
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(encoding_dim,))
    return z_mean + K.exp(z_log_sigma) * epsilon

Network structure is similar to the autoencoder. The main difference is in the middle of the network: z_mean and z_log_sigma. These layers encode a mean and log(std) that determine the pdf that we draw the encoding in the latent space from. 



The "Lambda" layer then draws a sample from that pdf, and z is the encoded signal in the latent space.

## input layer is full time series of length nt

In [16]:
# input layer is full time series of length nt
inputs = Input((NWL,))

## encoder hidden layers

In [17]:
# encoder hidden layers
encoded = Dense(1024)(inputs) 
encoded = LeakyReLU(alpha=0.2)(encoded)
encoded = Dense(64)(encoded)
encoded = LeakyReLU(alpha=0.2)(encoded)
encoded = Dense(32)(encoded)
encoded = LeakyReLU(alpha=0.2)(encoded)
z_mean = Dense(encoding_dim)(encoded)
z_log_sigma = Dense(encoding_dim)(encoded)
z = Lambda(sampling,output_shape=(encoding_dim,))([z_mean,z_log_sigma])

## decoder hidden layers

In [18]:
decoded = Dense(32)(z)
decoded = LeakyReLU(alpha=0.2)(decoded)
decoded = Dense(64)(decoded)
decoded = LeakyReLU(alpha=0.2)(decoded)
decoded = Dense(1024)(decoded)
decoded = LeakyReLU(alpha=0.2)(decoded)
# output layer is same length as input
outputs = Dense(NWL,activation='tanh')(decoded)

## consolidate to define autoencoder model inputs and outputs

In [19]:
# consolidate to define autoencoder model inputs and outputs
vae = Model(inputs=inputs, outputs=outputs)

In [20]:
# specify encoder and decoder model for easy encoding and decoding later
encoder = Model(inputs=inputs, outputs=[z_mean,z_log_sigma,z],name='encoder')
# create a placeholder for an encoded input
encoded_input = Input(shape=(encoding_dim,))

In [21]:
# retrieve the last layers of the autoencoder model
decoded_output = vae.layers[-7](encoded_input)
decoded_output = vae.layers[-6](decoded_output)
decoded_output = vae.layers[-5](decoded_output)
decoded_output = vae.layers[-4](decoded_output)
decoded_output = vae.layers[-3](decoded_output)
decoded_output = vae.layers[-2](decoded_output)
decoded_output = vae.layers[-1](decoded_output)
# create the decoder model
decoder = Model(inputs=encoded_input, outputs=decoded_output, name='decoder')

In [22]:
print('Full autoencoder')
print(vae.summary())

Full autoencoder
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10000)]      0           []                               
                                                                                                  
 dense (Dense)                  (None, 1024)         10241024    ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 1024)         0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 64)           65600       ['leaky_re_lu[0][0]']            
                                                                             

## VAE loss

The loss function is another key difference between standard autoencoders and variational autoencoders. A standard autoencoder simply minimizes reconstruction loss. 

- A variational autoencoder minimizes both reconstruction loss and the KL divergence. 

- The KL divergence is a measure of how much two probability distributions differ. 


Minimizing the KL divergence here means that we are encouraging the latent space encodings to have a normal distribution. The regularization parameter balances between reconstruction loss and enforcing a normal distribution in the latent space.

In [23]:
# specify loss
# regularization balances signal reconstruction with
# a Gaussian distribution in the latent space 
# regularization = 10
regularization = 10

def vae_loss(input_img, output):
    # compute the average MSE error, then scale it up, ie. simply sum on all axes
    reconstruction_loss = K.sum(K.square(output-input_img))
    kl_loss = - 0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.square(K.exp(z_log_sigma)), axis=-1)
    # return the average loss over all images in batch
    total_loss = K.mean(reconstruction_loss + regularization*kl_loss)    
    return total_loss

vae.compile(optimizer='adam', loss=vae_loss, metrics=['mse'],experimental_run_tf_function=False,run_eagerly=False)
vae.run_eagerly = False

## train variational autoencoder

In [ ]:
# train variational autoencoder
results = vae.fit(x_train,x_train,
                      shuffle=True,
                      batch_size = batch_size, 
                      epochs = epochs,
                      validation_data = (x_test,x_test),
                      callbacks = callbacks)

Train on 440 samples, validate on 110 samples
Epoch 1/50


2023-03-23 18:37:36.916056: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/dense_7/kernel/v/Assign' id:690 op device:{requested: '', assigned: ''} def:{{{node training/Adam/dense_7/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/dense_7/kernel/v, training/Adam/dense_7/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


440/440 [==============================] - ETA: 0s - loss: 45.9426 - mean_squared_error: 2.9866e-04
Epoch 1: val_loss improved from inf to 9.48923, saving model to modelNN_VAE_V0.h5


/Users/sylvie/mambaforge/envs/fidle2023/lib/python3.9/site-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2023-03-23 18:37:38.802991: W tensorflow/c/c_api.cc:300] Operation '{name:'loss/mul' id:288 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/dense_8_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


440/440 [==============================] - 4s 8ms/sample - loss: 45.9426 - mean_squared_error: 2.9866e-04 - val_loss: 9.4892 - val_mean_squared_error: 5.4343e-05
Epoch 2/50
440/440 [==============================] - ETA: 0s - loss: 8.7704 - mean_squared_error: 4.8855e-05
Epoch 2: val_loss improved from 9.48923 to 2.28977, saving model to modelNN_VAE_V0.h5
440/440 [==============================] - 4s 9ms/sample - loss: 8.7704 - mean_squared_error: 4.8855e-05 - val_loss: 2.2898 - val_mean_squared_error: 1.7133e-05
Epoch 3/50
440/440 [==============================] - ETA: 0s - loss: 0.3670 - mean_squared_error: 1.6544e-05
Epoch 3: val_loss improved from 2.28977 to -1.65643, saving model to modelNN_VAE_V0.h5
440/440 [==============================] - 5s 10ms/sample - loss: 0.3670 - mean_squared_error: 1.6544e-05 - val_loss: -1.6564 - val_mean_squared_error: 1.1040e-05
Epoch 4/50
440/440 [==============================] - ETA: 0s - loss: -1.4056 - mean_squared_error: 1.3328e-05
Epoch 4: v

In [ ]:
# QC training and validation curves (should follow eachother)
plt.figure(figsize=(8,8))
plt.plot(results.history['val_loss'], label='val')
plt.plot(results.history['loss'], label='train')
plt.xlabel('epoch index')
plt.ylabel('loss value (MSE)')
plt.legend()
plt.show()

## First, let's check to see how well the encoder is working.

In [ ]:
encoded_test = np.array(encoder.predict(x_test))
vae_test = vae.predict(x_test)

In [ ]:
plt.plot(x_test[2],label='true signal')
plt.plot(vae_test[2],label='encoded-decoded signal',alpha=0.5)
plt.legend()
plt.show()

## Now let's look at the distribution of samples in the latent space, to see how gaussian it is.

In [ ]:
import matplotlib.mlab as mlab
n,bins,patches=plt.hist(encoded_test[2,:,:].flatten(),bins=100,density=True)
plt.plot(bins,scipy.stats.norm.pdf(bins,0,1))
plt.show()

## Now let's see how the decoded signal depends on the latent vector.

Interestingly, the autoencoder has learned to use the latent vector (value in this case, since we specified an encoding dimension of 1) as a proxy for phase. As the latent value changes, the phase of the decoded signal changes. Latent values near zero reproduce a sine wave well, while values far from zero produce signals that aren't exactly sinusoidal.